In [1]:
using Pkg, Revise
Pkg.activate(".")

  Activating environment at `~/Desktop/SelectivelyAdaptiveLasso/Project.toml`


In [2]:
import SelectivelyAdaptiveLasso as SAL

┌ Info: Precompiling SelectivelyAdaptiveLasso [52f33b2b-83b2-4cfe-8aa4-4763af86ef4f]
└ @ Base loading.jl:1342


In [3]:
import Statistics: mean
import Distributions as Dist
import LinearAlgebra: diagm

n, p = 5000, 10

X_dist = Dist.reshape(
    Dist.MvNormal(zeros(p), 0.1ones(p,p) + 0.9diagm(ones(p))), 
    (1,p)
)

X = vcat(rand(X_dist, n)...);
# Y = rand(n)
Y = 5X[:,1] + X[:,2].^2 - 3X[:,3].*X[:,4]
Y = Y .- mean(Y)

n_val = Int(round(n/2))
X_val, Y_val = X[1:n_val,:], Y[1:n_val]
X, Y = X[(n_val+1):end,:], Y[(n_val+1):end];

In [9]:
sal_spec = SAL.SALSpec(
    max_iter=40,
    λ_max = 0.02,
    m_subsample=400,
    n_subsample=400,
)
sal_fit, log = SAL.fit(
    sal_spec, 
    X, Y, 
    X_val=X_val, Y_val=Y_val
)
loss, loss_val, _, _ = log;

(18.850426792376467, 16.222242585626624, 0.02, Set([1]))
(18.851653834856858, 16.229862813853586, 0.02, Set([4]))
(18.85019012203149, 16.90558876435196, 0.02, Set([6, 1]))
(18.850230129047763, 16.90616814119545, 0.02, Set([3]))
(18.837747589553427, 16.908212741695216, 0.02, Set([10]))
(18.808117570684782, 19.061064933272426, 0.02, Set([10, 1]))
(18.79304337661477, 22.31000526167395, 0.02, Set([3, 1]))
(18.7918248026099, 22.139761593142875, 0.02, Set([9]))
(18.161313530249217, 22.320869855462842, 0.02, Set([2]))
(18.139811031361297, 25.269178774851063, 0.02, Set([5, 8, 1]))
(17.914265239304417, 25.209808876069328, 0.02, Set([4, 3]))
(17.91426901986092, 25.211383634498663, 0.02, Set([2, 1]))
(17.914340754784302, 25.213685917799, 0.02, Set([2, 10, 1]))
(17.840248591869084, 25.59550595290499, 0.02, Set([5, 4, 1]))
(17.840385926701806, 25.601628680296546, 0.02, Set([4, 2, 3]))
(17.84046931270551, 25.60785856111278, 0.02, Set([6, 7, 10]))
(17.840898384330835, 25.61021639548203, 0.02, Set([4,

In [12]:
Ŷ = SAL.predict(sal_fit, X_val)
mean((Y_val - Ŷ).^2)

15.371760625259258

In [13]:
using XGBoost

In [14]:
bst = xgboost(X, 100, label = Y, eta = 0.1, max_depth = 4)
Ŷ = predict(bst, X_val)

[1]	train-rmse:5.667721
[2]	train-rmse:5.306073
[3]	train-rmse:4.987460
[4]	train-rmse:4.705669
[5]	train-rmse:4.462399
[6]	train-rmse:4.246899
[7]	train-rmse:4.056801
[8]	train-rmse:3.888477
[9]	train-rmse:3.748150
[10]	train-rmse:3.620879
[11]	train-rmse:3.513781
[12]	train-rmse:3.416229
[13]	train-rmse:3.314048
[14]	train-rmse:3.229013
[15]	train-rmse:3.148196
[16]	train-rmse:3.089708
[17]	train-rmse:3.038271
[18]	train-rmse:2.968949
[19]	train-rmse:2.920964
[20]	train-rmse:2.856219
[21]	train-rmse:2.822420
[22]	train-rmse:2.780844
[23]	train-rmse:2.751428
[24]	train-rmse:2.726351
[25]	train-rmse:2.691620
[26]	train-rmse:2.619180
[27]	train-rmse:2.601338
[28]	train-rmse:2.487532
[29]	train-rmse:2.472124
[30]	train-rmse:2.447840
[31]	train-rmse:2.431276
[32]	train-rmse:2.413248
[33]	train-rmse:2.395501
[34]	train-rmse:2.374310
[35]	train-rmse:2.359915
[36]	train-rmse:2.341735
[37]	train-rmse:2.324453
[38]	train-rmse:2.311752
[39]	train-rmse:2.214253
[40]	train-rmse:2.180028
[41]	trai

2500-element Vector{Float32}:
  2.125951
 -0.8188777
  1.3464571
  1.9144443
  1.0231552
 -1.1149218
 -5.366417
  1.8568602
 -4.266981
 -5.026776
  1.9554496
  1.887693
 -9.90477
  ⋮
  1.2130193
 -5.3429523
 -5.8551846
 -2.3047516
 -4.913264
  6.933373
 -1.7384822
 -4.6576014
  0.6591926
  3.2764697
  7.2495575
  1.667335

In [15]:
mean((Y_val - Ŷ).^2)

2.850314579614823